In [1]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import re
import pandas as pd
import numpy as np

In [4]:
class Preprocessor_kor:
    def __init__(self, data, stop_words):
        self.data = data
        self.stop_w = stop_words
        self.X_data = []
    
    # 데이터 확인용
    def print_data(self):
        print(self.data)
    
    # 데이터의 NaN값을 지우는 메소드
    def remove_null(self):
        self.data = self.data.dropna(how = 'any')
        
    # 중복된 데이터를 지우는 메소드
    def remove_duplication(self, subset):
        self.data = self.data.drop_duplicates(subset = [subset])
        
    # 데이터 체크 메소드
    def ckeck_data(self):
        print(f'데이터 길이: \n{len(self.data)}')
        print()
        print(f'결측치 : \n{self.data.isnull().sum()}')
        print()
        print(f'중복 데이터 제외 : \n{self.data.nunique()}')
     
    # 불용어 추가 메소드
    def add_stop_words(self, stop_word):
        self.stop_w.add(stop_wrod)
       
    # 전처리 메소드(매개변수로 X_data로 학습할 열을 받음)
    def preprocessor(self, X):
        self.data['clean_X'] = self.data[X].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
        self.data['clean_X'] = self.data['clean_X'].str.replace('^ +', '')
        self.data['clean_X'] = self.data['clean_X'].replace('', np.nan)
        self.remove_null()
        okt = Okt()
        for i in self.data['clean_X']: 
            tk_d = okt.morphs(i) 
            end_d = [w for w in tk_d if not w in self.stop_w] 
            self.X_data.append(' '.join(end_d))
      
    # 분류기 메소드(매개변수로 Y_data로 학습할 열, Y_data가 문자열인지, Y_data가 이중분류 데이터인지를 받음)
    def data_classificator(self, Y_column, isstr = True, isbin = True): 
        if isstr:
            self.data['encoder_y'] = LabelEncoder().fit_transform(self.data[Y_column])
            if isbin:
                Y = np.array(self.data['encoder_y'])
            else:
                Y = to_categorical(self.data['encoder_y'])
        else:
            if isbin:
                Y = np.array(self.data[Y_column])
            else:
                Y = to_categorical(self.data[Y_column])
        X = np.array(self.X_data)
        
        x_data, tt_x, y_data, tt_y = train_test_split(X, Y, test_size = 0.3, random_state = 0)
        t_x, v_x, t_y, v_y = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)
        
        tk = Tokenizer()
        tk.fit_on_texts(t_x)
        n = len([d for d in sorted(list(tk.word_counts.items()), key = lambda x: x[1]) if d[1] > 4]) + 1
        
        token = Tokenizer(n)
        token.fit_on_texts(t_x)
        
        token_train_x = token.texts_to_sequences(t_x)
        token_test_x = token.texts_to_sequences(tt_x)
        token_val_x = token.texts_to_sequences(v_x)
        
        drop_train = [index for index, sentence in enumerate(token_train_x) if len(sentence) < 1]
        drop_test = [index for index, sentence in enumerate(token_test_x) if len(sentence) < 1]
        drop_val = [index for index, sentence in enumerate(token_val_x) if len(sentence) < 1]
        
        token_train_x = np.delete(token_train_x, drop_train, axis = 0)
        t_y = np.delete(t_y, drop_train, axis = 0)
        token_test_x = np.delete(token_test_x, drop_test, axis = 0)
        tt_y = np.delete(tt_y, drop_test, axis = 0)
        token_val_x = np.delete(token_val_x, drop_val, axis = 0)
        v_y = np.delete(v_y, drop_val, axis = 0)
        
        w_l = len(pad_sequences(token_train_x)[0])
        train_inputs = pad_sequences(token_train_x, maxlen = w_l)
        test_inputs = pad_sequences(token_test_x, maxlen = w_l)
        val_inputs = pad_sequences(token_val_x, maxlen = w_l)
        train_outputs = t_y
        test_outputs = tt_y
        val_outputs = v_y
        
        return train_inputs, test_inputs, val_inputs, train_outputs, test_outputs, val_outputs, n, token, w_l